# Python and Data Analysis 2 - Cleaning, Preparing, and Organizing Data

**Goal:** The goal of this project is to learn to prepare data for analysis using Pandas.

**Description:** Data often needs to be organized, combined, or cleaned before performing analysis. This project explains a few key ways to accomplish this.

## 2A: Preparing and Cleaning Data

Data *cleaning* refers to the process of ensuring our data is correct and complete. If data is not cleaned, our analysis could produce incorrect results based on corrupt/garbage information. *Preparation* refers to getting our data ready for analysis by putting it into a format we can easily understand.

### Removing Garbage Data

*Garbage data* depends on the type of data we are representing. One common example is if we have a DataFrame with inconsistent types in its columns.

In [ ]:
import pandas as pd

df = pd.DataFrame({'x': [1,'more garbage',3,4,5,6,7,8,9,10],
                  'y': [1,4,9,16,25,36,49,64,'garbage',100]})
print(df)

Clearly, we want `x` and `y` to contain numerical values. Therefore rows 1 and 8 contain garbage data. We can try converting them to numbers to fix this.

In [ ]:
df = df.apply(pd.to_numeric, errors='coerce') # Go through the DataFrame and force every value to be a number
print(df.dtypes)
print(df)

Notice the strings have been replaced `NaN`. This is a special value considered to be a `float`, so it is consistent with the other numerical values in its respective column. However, when NaN is involved in a calculation, it results in NaN output. Therefore, we typically want to remove these values.

In [ ]:
# NaN + 4 = NaN
print(df.iloc[1,0] + df.iloc[1,1]) # Add the x value from row 1 (NaN) to the y value from row 1 (4)

We can remove `NaN` values in two ways. If we want to completely delete the row containing the NaN, we can use `dropna()`. If we want to replace the NaN with some constant value, we can use `fillna(replacement_value)`. The first example removes rows with NaN, and the second example replaces the NaNs with the value 0. 

In [ ]:
df1 = df.dropna()
# print("Dropping NaNs")
# print(df1)

df2 = df.fillna(0)
print("\nFilling NaNs")
print(df2)

### Preparing Data

#### Dropping Columns
When getting data ready for analysis, we might want to select subsections of our data and crop out the rest. For example, let's say we are interested in the `date`, `close` and `volume` for Microsoft stock. Using the techniques discussed earlier, we can do the following.

In [ ]:
df = pd.read_csv('MSFT.csv')
df = df[['date', 'close', 'volume']]
print(df.head())

Alternatively, if there were some columns which we were just not interested in, we could `drop` those and leave the rest. Note: we need `axis=1` to tell Pandas to drop columns and not rows.

In [ ]:
df1 = pd.read_csv('MSFT.csv')
df1 = df1.drop(['close'], axis=1)
print(df1.head())

#### Changing Data Types
After collecting the data we want, we sometimes need to change the type of a column. A common example is converting `date` from `string` to `datetime`.

In [ ]:
df['date'] = pd.to_datetime(df['date']) # Convert 'date' column from string to datetime
print(df['date'].head()) # Display the first few rows of the 'date' column

#### Renaming Columns
We can easily rename our columns with `rename`. We supply a dictionary, mapping the old names to new ones.

In [ ]:
df = df.rename(columns={'date': 'Date', 'close': 'Closing Price', 'volume': 'Trading Volume'})
print(df.head())

#### Changing Index
Finally, we might want to relabel our index, by setting its values to be the `Date` column. This is useful if we frequently want to access rows by `Date`.

In [ ]:
df = df.set_index('Date')
print(df.head())


In [ ]:
print(df.loc["1986-03-17"])
print(df.loc["1986-03-13"])


Note, the new index is the old `Date` column, which has been removed. We can no longer access `Date` using `df['Date']` because it is no longer considered a column. We would instead access the index using `df.index`, `df.index.values`, or `df.index.values.tolist()`.

**Challenge**: Reload the MSFT CSV into a variable called `df1`, then drop the open and volume columns from it, convert the date column to datetime, then use the date column as the index. **Important**: don't name the new dataframe `df`.

In [ ]:
df3 = pd.read_csv('MSFT.csv')
df3 = df3.drop(['open', 'volume'], axis=1)
# print(df3.head())

df3['date'] = pd.to_datetime(df3['date'])
# print(df3.head())
df3 = df3.set_index('date')
print(df3.head())

print(df3.loc['1986-03-17'])

# df1 = pd.read_csv('MSFT.csv')
# df1 = df1.drop(['open', 'volume'], axis=1)
# df1['date'] = pd.to_datetime(df1['date'])
# df1 = df1.set_index('date')
# print(df1.head())

### Sorting Data

Finally, we might want to sort our data by one or more columns. Lets say we want to sort the data in increasing order by `volume`.

We use the syntax `df-name.sort_values()` which takes the parameters:
 - `by=`: The column name or column list to sort by
 - `ascending=`: Whether you want to sort in increasing (True) or decreasing (False) order

In [ ]:
msft = pd.read_csv('MSFT.csv')
msft = msft.sort_values(by=['volume', 'close'], ascending=True)
print(msft)

If we wanted to sort by multiple columns, we pass a list into the `by=` parameter. For example, to sort by `volume` and then `close`, we would do `msft.sort_values(by=['volume', 'close'], ascending=True)`.

## 2B: Combining DataFrames

Sometimes, data is scattered across multiple DataFrames. In that case, we need to be able to join DataFrames together. There are several ways to do this in Pandas, but this tutorial will cover `merge`. Suppose we have the following two DataFrames.

In [ ]:
classes = pd.DataFrame({
    'class': [1,2,3,4,5],
    'teacher': ['Bill Ryan', 'Sue Kim', 'Arun Gupta', 'Darnell Williams', 'Emily Coles'],
})

teachers = pd.DataFrame({
    'teacher': ['Sue Kim', 'Arun Gupta', 'Emily Coles', 'Darnell Williams', 'Ben Smith', 'Nicole Ang'],
    'age': [43, 56, 32, 44, 32, 53],
    'experience': [4, 5, 2, 6, 1, 4],
})

print("Classes")
print(classes)
print("\nTeachers")
print(teachers)

The `classes` DataFrame contains information about 5 classes and the teachers that teach them. The `teachers` DataFrame contains information about each teacher: their age, and years of experience. We want to obtain one DataFrame, with information about class, teacher, age, and experience. There are four ways to do this. 

When joining DataFrames, we visualize the operation as adding the *right* DataFrame to the *left* one. We join based on a column that contains the same data between the two DataFrames. In this case, the shared piece of data is the teacher.

The syntax is `left_df.merge(right_df, on='', how='')`.
 - `on=''`: Replace the quotes with the common column between the two DataFrames
 - `how=''`: Replace the quotes with the type of join: `'left'`, `'right'`, `'outer'`, `'inner'`.

### Left Join

A *left join* keeps all the rows in the left DataFrame, and adds entries from the right DataFrame where the `on` column matches.

In [ ]:
combined = classes.merge(teachers, on='teacher', how='left')
print(combined)

The ages and experience for Sue, Arun, Darnell, and Emily have been added to the left DataFrame. Bill Ryan remains because he was in the left DataFrame, but does not have an age or experience because he is not present in the right DataFrame.

### Right Join

A *right join* keeps all the rows in the right DataFrame, and adds entries from the left DataFrame where the `on` column matches.

In [ ]:
combined = classes.merge(teachers, on='teacher', how='right')
print(combined)

As before, the ages and experience for Sue, Arun, Darnell, and Emily have been added. Bill Ryan was not in the right DataFrame, so his name is not present, and his class has disappeared as well. Ben Smith and Nicole Ang were present in the right DataFrame, so they are included, but they do not have an associated class.

### Inner Join

An *inner join* keeps all the rows where the `on` column matches - in other words, where the `teacher` is present in both DataFrames.

In [ ]:
combined = classes.merge(teachers, on='teacher', how='inner')
print(combined)

The ages and experience for Sue, Arun, Darnell, and Emily are included because they are present in both DataFrames. Bill Ryan is only present in the left DataFrame, so he is not included. Ben Smith and Nicole Ang are only present in the right DataFrame, so they are not included.

### Outer Join

An *outer join* keeps all entries from both DataFrames. Where values don't exist, it fills the cell with `NaN`.

In [ ]:
combined = classes.merge(teachers, on='teacher', how='outer')
print(combined)

Every teacher is included in this combination. If they don't have an age or experience (Bill Ryan) it is filled with NaN. If they don't have a class (Ben Smith, Nicole Ang), it is filled with NaN. Try and think about which types of joins would be useful in different scenarios.

## 2C: Grouping Data

Finally, if we have a large DataFrame, we might be interested organizing the data into groups. We can do this easily using `groupby`. In the following example, we create dataframes for Microsoft, Apple, Amazon, and Google stock prices. Then we `append` them together into one large DataFrame. 

In [ ]:
stock_names = ['MSFT', 'AAPL', 'AMZN', 'GOOG']

df = pd.DataFrame()

df_list = []  # create a list for saving DataFrame

for stock_name in stock_names:
    stock_df = pd.read_csv(f'{stock_name}.csv')  # read CSV
    stock_df['name'] = stock_name  # add column with company's name
    df_list.append(stock_df)  # add to the list

df = pd.concat(df_list, ignore_index=True) 
# ignore_index=True for updating indices
print(df)

Let's say `df` was the only DataFrame we had. Without knowing where each stock's price data begins and ends, there is no way to separate data by stock (the `name` column). Instead, we use `groupby` to create an object that stores the grouped DataFrames. Here, we want to group the rows in the DataFrame by `name`.

In [ ]:
groups = df.groupby('name') # Creates a grouping of the DataFrame by column 'name'
print(groups)

We cannot display `groups` directly, because it is not a DataFrame. Instead, we can access specific groups stored in `groups`. The following code gets all the rows in the group `'MSFT'`, returns them as a DataFrame, and stores them in the variable `msft_data`. The resulting DataFrame contains price information for only Microsoft's stock.

In [ ]:
msft_data = groups.get_group('AAPL')
print(msft_data)

To practice, try getting the data for Apple, Google, and Amazon from `groups`. We don't have to group by `name`, but we can also group by other columns as well. For example, if we wanted to compare data across all stocks on a given date, we could `groupby('date')`.

In [ ]:
dates_groups = df.groupby('date') # A grouping of the DataFrame by date

To view the stock prices on a given day (for example January 3, 2018), we can `get_group` for the day we are interested in.

In [ ]:
target_data = dates_groups.get_group('2018-01-03')
print(target_data)

**Challenge**: Find the most common volume amount across all four stocks using `groupby`, then print the rows in the dataframe with that volume using `get_group`.

In [ ]:
t = df.groupby('volume')
# print(t.get_group(7900))
# print(t.groups.items())
# Sort based on the number of values in each group
most_common = max(t.groups.items(), key=lambda item: len(item[1]))
# print(most_common)
# print(t.get_group(most_common[0]))